## boomdiff
Documentation for Milestone 2
Group Members: Minhaun Li, Oksana Makarova, Timothy Williamson, Kevin Hare

## Introduction
This software package `Boomeraang_optimizer` implements an optimization of a user-supplied or pre-set objective function. Many scientific and social scientific fields rely on probabilistic and statistical methodologies that fundamentally concern the optimization of some function, often called an objective function or a cost function. Intuitively, these methods seek to find the best fit of some function given observed data. Two common point estimators in data-driven fields are Maximum Likelihood Estimator (MLE) from Frequentist's view and Maximum A Posterior (MAP) from Bayesian's view. In each case, the optimum points are identified by the stationary condition (first order derivative equals 0) and convexity check by higher order derivatives. Generally, a major branch of modern optimization methods, like Stochastic Gradient Descent (SGD) and Broyden–Fletcher–Goldfarb–Shanno algorithm(BFGS), are established via the first or higher order gradient of target function, thus heavily relies on an efficient gradient computation.

Auto Differentiation (AD) is one highly effective method for calculation of the gradient function at some point. The method, which balances the efficiency of numeric computation and the precision of symbolic derivatives, is commonly used for optimization applications. Our library solves the optimization problem described above via gradient descent, which is implemented on top of a forward-mode autodifferentiation. The advance of this method is that AD can effectively and efficiently compute the Jacobian matrix. For the optimization problems we consider, the multidimensional nature of the challenge necessarily lends itself to the use of AD.

## Background
At it’s heart, automatic differentiation (AD) seeks to calculate the derivative of some function, and evaluate both the function and the derivative, at a given point by iteratively applying the chain rule to a composition of elementary functions whose derivatives are well-known. This application of the [chain rule](https://en.wikipedia.org/wiki/Chain_rule) is essential. By viewing a more complex function as simply the composition of many elementary functions, the calculation of the derivative becomes a series of steps, starting from the innermost function in the forward mode of AD. In each step, the chain rule is applied. For $z(y(x))$:

$$ \frac{dz}{dx} = \frac{dz}{dy} \frac{dy}{dx} $$

Importantly, because $z(y)$ will be an elementary function (e.g. addition, subtraction, sine, cosine), its derivative can be easily calculated. As we have begun at the innermost derivative, that function $\frac{dy}{dx}$ is known and can be used to iteratively calculate the derivative of the composition. For the simple example above, there are only two elementary operations, but this method can be extended to cover many elementary operations. One only needs to keep track of the derivative of each ‘running’ piece.

The method can be implemented through a graph structure (see below for a simple example), with each node in the graph, which represents a single elementary operation. This computational graph is especially important when the function of interest relies on multiple elements (e.g. $f(x,y) = x^2 + \sin(y)$). One particular advantage of the elementary operations in this method is that each type of operation has a known derivative that can be calculated systematically and efficiently. At each step in the forward mode, the algorithm only needs to maintain the status of the derivative (potentially multiple partial derivatives in the case of multiple elements) as well as the current value of the function.

These storage requirements and iterative nature take advantage of a computer’s ability to store many values and perform many simple operations very quickly. One challenge for computers in calculating complex, symbolic derivatives is that symbolic differentiation may lead to enormous equations and syntactical rules that are highly complex to apply. For humans with a pen and paper, the AD approach may take too much time. The number of calculations, albeit simple, would certainly overwhelm the ability of most people to simply calculate the symbolic derivatives and implement a single evaluation. That concern is severely attenuated by computers. Additionally, AD is superior to the finite differences method of differentiation due to the fact that it is able to keep track of derivatives and functions at the level of machine precision. In scientific applications, this feature is incredibly important, as the sensitive systems measured or engineered will not be successful with only generalities.

## Installation of boomdiff
Below are instructions for the download and installation of the *boomdiff* package. The instructions below are all designed to be run on a command-line interface (Windows, macOS, Linux). Please note that steps (2) and (3) assume that the user has installed [Python](https://www.python.org/). If the user has not installed Python, please see instructions [below](#installation_py)

1. **Download package**: 

    The boomdiff package is available at the GitHub address (https://github.com/team-boomeraang/cs107-FinalProject). To download, navigate on the command line to the desired installation location and run:

    `git clone https://github.com/team-boomeraang/cs107-FinalProject.git`
    
    
2. **Installation of dependencies**: 

    There are two ways to install dependencies, depending on the package manager used. If using `pip`, then run:

    `pip install -r requirements.txt`
    
    Else, if using Conda or Miniconda for package management, run:
    
    `conda install --file requirements.txt`
    
    Finally, if the desired use is to use `pip` to install the package through Conda or Miniconda, run the following sequence of commands:
    
    `conda install pip`
    
    `which pip`
    
    Ensure that the directory of pip to be used falls below 'anaconda' or 'miniconda' in the directory structure.
    
    `pip install -r requirements.txt`
    
3. **Set-up and install packages**: 
    
    Next, the `setup.py` file must be run to install boomdiff [boomdiff_optimizer not currently available, see Future Features section below]. Navigate to the newly cloned directory and run:

    `python setup.py install`
    

<a id='installation_py'></a>
### Installation of Python
We recommend two possible methods for installation of Python.


1. [Anaconda](https://www.anaconda.com/products/individual) is a package manager that can install Python as well as a variety of common packages (e.g. NumPy, Pandas, scipy). If installed with Anaconda, the instructions below describing `conda install` will be most relevant.


2. Python can also be installed directly (see [here](https://www.python.org/downloads/)). This installation mode will require use of an outside text editor, but may offer more flexibility for installation. If installed via the Python documentation directly, please follow instructions for `pip`.

## Use of boomdiff
*boomdiff* operates as an object oriented program. A brief introduction to how the program might be operated follows. To follow this documentation, please first ensure that *boomdiff* has been installed.

For the purposes of this demonstration, consider first the simple linear function $f(x) = 3x + 4$, and subsequently the more complex sinusoidal function $f(x) = sin(x^2)$. Please note that at this time, *boomdiff* is only able to work on scalar functions of one variable. This functionality, and how it interacts with the program execution, will be explained further below.

In [1]:
# Import boomdiff library
from boomdiff import AD

#### Example 0: Instantiate a variable 

As a forward AD mode, we mostly have to instantiate input variables with value and derivative values as a starting point. This process can be easily done with `x1 = AD(*value,**derivate_dict)`. Value is a necessary argument and derivative dictionary are defaulted to be `{'x1': 1}`, you can set as you like. Currently, we only support single input, single scalar output.

The value of the variable will be stored in an attribute `func_val` and the partial derivative dictionary (for now, only one key-value pair) will be stored as `partial_dict`. Then you can use this defined varaible for constructing following complex functions.

In [2]:
# Step1: Instantiate a variable called x1, with value 10, derivative to be default 1
x1 = AD(10)

# Step2: Demonstrate the information
print(x1)

10 ({'x1': 1})


You can name the variable and set the derivative value as you like. For a better practive, we recommend you set the variable name and the key string in the derevative dictionary to be the same. This is important when you use it for multivariant functions (not supported at the moment).

In [3]:
# Step1: Instantiate a variable called a, with value 4.9, derivative to be 3.4
# For a better practice 
a = AD(4.9, {'a': 3.4})

# Step2: Demonstrate the information
print(a)

4.9 ({'a': 3.4})


#### Example 1: $f(x) = 3x + 4$

We use this example as a demonstration of overloaded primary function in our package. Currently, we support `+`(add), `-`(subtract & negation), `*`(multiply), `/`(divide) and `**`(power)

We will evaluate this function and its derivative at the point $x = 10$. From analytical derivation, we can ascertain that $f(10) = 34$ and $f'(10) = 3$. The tutorial below demonstrates how to calculate this result using *boomdiff*. Before beginning, one important note is that because *boomdiff* only supports scalar functions of one variable, the default behavior is to label the variable `x1`. While the variable name may be any variable name supported by Python, *boomdiff* will report a derivative associated with `x1` unless otherwise specified.

The value of the function will be stored in an attribute `func_val` and the partial derivative dictionary (for now, only one key-value pair) will be stored as `partial_dict`.

In [4]:
# Step 1: Instantiate AD object, as a variable x, at evaluation point (10)
x = AD(10, {'x': 1})

# Step 2: Construct function f(x) = 3x + 4
f = 3*x + 4

# Step 3: Evaluate function and derivative
print(f.func_val)
print(f.partial_dict)

34
{'x': 3}


#### Example 2: $f(x) = \sin(x^2)$

We use this example as a demonstration of static method primary functions in our package. Currently, we support `AD.sin()`(sine), `AD.cos()`(cosine), `AD.tan()`(tangent), `AD.log()`(natural log) and `AD.exp()`(exponential). 

We will evaluate this function and its derivative at the value 3. Analytically, the derivative of this function, $f'(x) = 2x \cos(2x)$. Unlike the first function, this one is not so easily evaluated simply by inspection. For *boomdiff*, however, this remains computationally easy. Additionally, in this example, we directly instantiate the function. Note that this functionality assumes that there is only one variable, and thus, $x + y$ will be treated as $x1 + x1$.

In [5]:
from boomdiff import AD
# Step 1: Instantiate AD object, as a variable x, at evaluation point (3)
x = AD(10, {'x': 1})

# Step 2: Instantiate function f = sin(x**2)
f = AD.sin(x**2)

# Step 2: Evaluate function and derivative
print(f.func_val)
print(f.partial_dict)

-0.5063656411097588
{'x': 17.24637744575368}


## Software organization

## Implementation

#### boomdiff
- Core data structures:
    - boomdiff is primarily implemented through an object-oriented class, AD. This class is essentially the object or function to be evaluated and differentiated.
    - While the actual object must be separately called (e.g. AD(2)), this can be wrapped into a single function line, as the class will be returned and operated on by other methods of the AD class.
- Core classes:
    - AD class: `AD(eval_pt, der_dict)`
        - Constructor arguments:
            - `eval_pt`: Point to evaluate function at. Expected behavior as float or int; required.
            - `der_dict`: Optional. Dictionary to support dictionary for object of multiple variables. Default behavior will be to assume that function is one variable (i.e. x1) and that its derivative is 1. Currently does not support multiple variables, though variable name passed to `der_dict` could, in theory, be any string.
        - Attributes:
            - `func_val`: value of function as a float; this will initally be the `eval_pt` passed at class initialization.
            - `partial_dict`: dictionary. This dictionary will store the partial derivatives. Each key corresponds to the variable (in a multiple variable function). Note that the multiple variable functionality has not been fully implemented and tested.
         - Elementary functions:
             - These methods are executed in two manners, depending on the function type. Functions which have Python built-ins have been overloaded. Other elementary functions have been implemented as class static methods. Please see below for greater details.
             - Overloaded:
                 - Addition (`+`)
                 - Subtraction (`-`)
                 - Multiplication (`*`)
                 - Power (`**`)
                 - Division (`/`)
                 - Negation (`-x`), where x is an instance of AD.
                 
             - Static methods:
                 - Sine (`AD.sin()`)
                 - Cosine (`AD.cos()`)
                 - Tangent (`AD.tan()`)
                 - Natural logarithm (`AD.log()`)
                 - Exponential (base e) (`AD.exp()`)
                 
        - Other methods:
            - `set_params(att, val)`: Sets parameter values outside of initializer. Note that if this is done after operations have been performed, may not be backwards compatible; preferable to set parameters with constructor. `att` argument must be one of `func_val` or `partial_dict` and must mirror behavior of `eval_pt` and `der_dict` arguments to constructor.
                 
- External dependencies:
    - [NumPy](https://numpy.org/)
- To be implemented:
    - Additional overload/static operations (e.g. sigmoid function, inverse trigonometric, hyperbolic  etc.)
    - Special methods for comparison: __eq__, __neq__, potentially including less than and greater than operators.
    - evaluate(): Will return the value of the function
    - derivatives(): Will return the partial derivative dictionary

## Future Feature1 -- Multivariant Input and output

We want to extend our library to support multivariant inputs and output. The `partial_dict` as a dictionary in the current class stuctures make the extension not very hard, and all we need to do is considering how to join and append dictionarys. We expect the usage to be like the following code fence:

```python
    from boomdiff import AD
    
    # Step1: instantiate multiple variables, x1, x2 here
    x1 = AD(10, {'x1': 1})
    x2 = AD(4.9, {'x2': 1})
    
    # Step2: Define a multivariant function, f = x1+x2
    f = x1 + x2
    
    # Step3: Show the information of f
    f.func_val
Out:14.9   
    
    # Now we have multiple partial derivatives in the dictionary
    f.partial_dict
Out:{'x1': 1, 'x2': 1}
```



## Future Feature2 -- Optimization library
For our future feature, we propose implementing an optimization library on top of `boomdiff`. This library will be tightly linked to `boomdiff`, but will offer additional functionality. In particular, the intended use of this library would be to optimize an objective function. That objective function is currently broadly defined, as we hope to supply the user with functionality. Additionally, we will offer multiple gradient-based methods of optimization including at least: gradient descent, coordinate descent (e.g. [random coordinate descent](https://en.wikipedia.org/wiki/Random_coordinate_descent)).

In terms of implementation, we intend for this library to be built primarily on `ObjectiveFunction` class, to which a user may directly instantiate some objective function, or build one from `boomdiff` objects. For example, a user may present the following code to perform maximum likelihood estimation on a Cauchy distribution, after observing some `x`, say 10 (assume the scale parameter is known).

```python
    from boomdiff import AD
    from obj_optimizer import ObjectiveFunction
    import numpy as np
    
    x_init = 0
    
    def f(x):
        return (1/np.pi)*(1/(1 + (10 - AD(x))**2)
    
    optimizer = ObjectiveFunction(f=f, x_init=x_init, method='gradient_descent')
    x_opt = optimizer.optimize()
```  

There are a few important considerations for this library. First, the user will need to be careful to specify each of the objects to be optimized as *boomdiff* objects. One potential direction, time permitting, would be to incorporate a parser into our functionality to improve the user's ability to call any function. For now, however, we will require inclusion of AD objects in the objective function. A second challenge will be incorporating data -- specifically tabular data. For example, in the above example, a user may want to calculate the maximum likelihood after receiving not one but one thousand `x` values. Doing so would preferably require definition of the problem in the log space. This will likely be done either through a `sum()` method, or by reliance on NumPy to calculate sums. One alternate approach would be to re-define some of our static methods in this separate class to mirror the functions from the *boomdiff* library. For example, by masking `log` as `log` from AD, this may reduce confusion for the user, allowing a call to `obj_optimizer.log()` in the definition of their `f(x)` as demonstrated above.